# Storing scraped articles

In [1]:
import numpy as np
import pandas as pd
import requests
import re
import sqlite3
import time
import json
import openai
import redis

In [2]:
r = redis.Redis("localhost", 5211, decode_responses=True)

In [3]:
db = sqlite3.connect("scraping.db")

In [4]:
cur = db.execute("select count(*) from article where content is null")
db.commit()
for row in cur:
    print(row)

(219401,)


In [6]:
n = 0
while True:
    if (data := r.rpop("to_store_newsam")) is None:
        print(f"{n} Nothing to store, idling...", end="\r")
        time.sleep(1)
    else:
        data = json.loads(data)
        
        if len(data["id"]) == 0:
            print("ERROR, NO ID")
            continue
        if len(data["content"]) == 0:
            print("ERROR, NO CONTENT")
            continue

        n += 1
        print(str(n) + "      ", end="\r")

        db.execute(f'update article set region = ?, theme = ?, content = ? where id = ?', (data["region"], data["theme"], data["content"], int(data["id"])))
        db.commit()